In [3]:
import csv
import pandas as pd
import numpy as np
from tpot import TPOTClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
import joblib

file_feature = "./csv/endometrium.csv"
file_train = "./csv/train.csv"
file_validate = "./csv/validation.csv"
file_test = "./csv/test.csv"

f = open(file_feature)
csv_f = csv.reader(f)
features = next(csv_f)
dataset = pd.read_csv(file_feature, names=features, usecols=range(1,6098), dtype=np.float64, skiprows=1, low_memory=False)
f = open(file_train)
csv_f = csv.reader(f)
features = next(csv_f)
dataset_train = pd.read_csv(file_train, names=features, usecols=range(1,1), dtype=np.float64, skiprows=1, low_memory=False)

with open('./csv/train.csv','r') as csvfile:
    reader = csv.reader(csvfile)
    train_list = [row[1] for row in reader]
with open('./csv/validation.csv','r') as csvfile:
    reader = csv.DictReader(csvfile)
    validation_list = [row['patient'] for row in reader]
with open('./csv/test.csv','r') as csvfile:
    reader = csv.DictReader(csvfile)
    test_list = [row['patient'] for row in reader]

dataset['outcome'] = pd.to_numeric(dataset['outcome'],errors='coerce')
array_OG = dataset.values
print(array_OG.shape)
train_list = train_list[1:]
validation_list = validation_list[0:]
test_list = test_list[0:]
#print(test_list)
#print(train_list)
#print(validation_list)

def cat_str(num_list):
    n_list = []
    for i in num_list:
        temp = i[12:]
        n_list.append(temp)
    n_list = [int(x) for x in n_list]
    return n_list

train_list = cat_str(train_list)
validation_list = cat_str(validation_list)
test_list = cat_str(test_list)

#print(train_list)
#print(validation_list)
#print(test_list)
#print(len(test_list))

train_feature = []
validate_feature = []
test_feature = []
count = 1
for i in range(len(array_OG)):
    num = i + 1
    if num in train_list:
        train_feature.append(array_OG[i])
    elif num in validation_list:
        validate_feature.append(array_OG[i])
    elif num in test_list:
        #print(count)
        count = count + 1
        test_feature.append(array_OG[i])
        #print(num)
        #print(array_OG[i,6096])
        
train_feature = np.array(train_feature)
validate_feature = np.array(validate_feature)
test_feature = np.array(test_feature)

train_feature = pd.DataFrame(train_feature)
train_feature.dropna(axis=1, thresh=2, inplace=True)
train_feature.dropna(how='all',thresh = 20,inplace=True)
train_feature = np.array(train_feature)
wh_inf = np.isinf(train_feature)
train_feature[wh_inf]=0
wh_nan = np.isnan(train_feature)
train_feature[wh_nan]=0

validate_feature = pd.DataFrame(validate_feature)
validate_feature.dropna(axis=1, thresh=2, inplace=True)
#validate_feature.dropna(how='all',thresh = 20,inplace=True)
validate_feature = np.array(validate_feature)
wh_inf = np.isinf(validate_feature)
validate_feature[wh_inf]=0
wh_nan = np.isnan(validate_feature)
validate_feature[wh_nan]=0

test_feature = pd.DataFrame(test_feature)
test_feature.dropna(axis=1, thresh=2, inplace=True)
#test_feature.dropna(how='all',thresh = 20,inplace=True)
test_feature = np.array(test_feature)
wh_inf = np.isinf(test_feature)
test_feature[wh_inf]=0
wh_nan = np.isnan(test_feature)
test_feature[wh_nan]=0

#only use image features
X_train = train_feature[:,:6093]
Y_train = train_feature[:,6093]
Y_train = Y_train.astype('int32')

X_validate = validate_feature[:,:6093]
Y_validate = validate_feature[:,6093]
Y_validate = Y_validate.astype('int32')

X_test = test_feature[:,:6093]
Y_test = test_feature[:,6093]
Y_test = Y_test.astype('int32')
seed = 7

np.random.seed(seed)
np.random.shuffle(X_train) 
np.random.seed(seed)
np.random.shuffle(Y_train)

print(len(Y_validate))

(849, 6097)
83


In [9]:
import csv
import pandas as pd
import numpy as np
from sklearn import model_selection
from tpot import TPOTClassifier
import tools

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostClassifier
#from pyearth import Earth
from sklearn.cross_decomposition import PLSRegression


from sklearn.feature_selection import SelectKBest

from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from skfeature.function.statistical_based import t_score
from skfeature.function.statistical_based import gini_index
from skfeature.function.similarity_based import fisher_score
from skfeature.function.similarity_based import reliefF

from sklearn.feature_selection import mutual_info_classif
from skfeature.function.information_theoretical_based import LCSI
from skfeature.function.information_theoretical_based import MIM
from skfeature.function.information_theoretical_based import MIFS
from skfeature.function.information_theoretical_based import MRMR
from skfeature.function.information_theoretical_based import CIFE
from skfeature.function.information_theoretical_based import JMI
from skfeature.function.information_theoretical_based import CMIM
from skfeature.function.information_theoretical_based import ICAP
from skfeature.function.information_theoretical_based import DISR

from scipy.stats import wilcoxon

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample

from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
# CLASSIFICATION METHODS
models = []
models.append(('GLM', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('DT', DecisionTreeClassifier()))
models.append(('BY', GaussianNB()))
models.append(('SVM', SVC()))
models.append(('BAG', BaggingClassifier()))
models.append(('NNet', MLPClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('BST', AdaBoostClassifier()))

seed = 7

In [10]:
#crossvalidation on trainset and select the best model on validation set, test on test set
import xlwt

num_fea = 100
sel = []
sel.append(('CHSQ', SelectKBest(chi2, k=num_fea)))
sel.append(('ANOVA', SelectKBest(f_classif, k=num_fea)))
sel.append(('TSCR', SelectKBest(t_score.t_score, k=num_fea)))
sel.append(('FSCR', SelectKBest(fisher_score.fisher_score, k=num_fea)))
sel.append(('RELF', SelectKBest(reliefF.reliefF, k=num_fea)))

output = open("endohand100.txt","w")
file = 'fea100.xls'


book = xlwt.Workbook()
sheet = book.add_sheet('train_avg_auc')
sheet_train = book.add_sheet('train_auc')
sheet_validate = book.add_sheet('validate_auc')
sheet_test = book.add_sheet('test_auc')


from sklearn.externals import joblib
#from imblearn.over_sampling import SMOTE
# 定义SMOTE模型，random_state相当于随机数种子的作用
kfold = model_selection.KFold(n_splits=5, random_state=seed)
r = 0
c = 0
for name, model in models:
    for kind, selection in sel:
        pipe = make_pipeline(MinMaxScaler(), selection, model)
        cv_results = model_selection.cross_val_score(pipe, X_train, Y_train, scoring='roc_auc', cv=kfold)
        
        sheet.write(r,c,cv_results.mean())

        print("###########################################")
        msg = "%s %s %s: %f (%f)\n" % ("TRAIN_AUC", kind, name, cv_results.mean(), cv_results.std())
        print(msg)
        output.write(msg)
        pipe.fit(X_train,Y_train)
        joblib.dump(pipe,'./handpkl/Endo'+name+kind+'100.pkl')
        
        Y_pred = pipe.predict(X_train)
        print("Accuracy: " + repr(accuracy_score(Y_train, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_train, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_train, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_train, Y_pred)))
        print("AUC: " + repr(roc_auc_score(Y_train, Y_pred)))
        print("Sensitivity: " + repr(recall_score(Y_train, Y_pred)))
        tn, fp, fn, tp = confusion_matrix(Y_train, Y_pred).ravel()
        print("Specificity: " + repr(tn / (tn + fp)))
        
        sheet_train.write(r,c,roc_auc_score(Y_train, Y_pred))

        Y_pred = pipe.predict(X_validate)
        print("Accuracy: " + repr(accuracy_score(Y_validate, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_validate, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_validate, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_validate, Y_pred)))
        print("AUC"+repr(roc_auc_score(Y_validate,Y_pred)))
        print("Sensitivity" + repr(recall_score(Y_validate,Y_pred)))
        tn,fp,fn,tp = confusion_matrix(Y_validate,Y_pred).ravel()
        print("Specificity" + repr(tn/(tn+fp)))
        
        sheet_validate.write(r,c,roc_auc_score(Y_validate,Y_pred))
        
        Y_pred = pipe.predict(X_test)
        print("Accuracy: " + repr(accuracy_score(Y_test, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_test, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_test, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_test, Y_pred)))
        print("AUC: " + repr(roc_auc_score(Y_test, Y_pred)))
        print("Sensitivity" + repr(recall_score(Y_test,Y_pred)))
        tn,fp,fn,tp = confusion_matrix(Y_test,Y_pred).ravel()
        print("Specificity" + repr(tn/(tn+fp)))
        
        sheet_test.write(r,c,roc_auc_score(Y_test,Y_pred))
        
        r = r + 1
    c = c + 1
    r = 0

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defau

###########################################
TRAIN_AUC CHSQ GLM: 0.794649 (0.034434)

Accuracy: 0.7484848484848485
Average Precision Score: 0.6431380206214643
Kappa: 0.49253344202764293
Hamming Loss: 0.2515151515151515
AUC: 0.7467329967329968
Sensitivity: 0.7297297297297297
Specificity: 0.7637362637362637
Accuracy: 0.7831325301204819
Average Precision Score: 0.6935813429155218
Kappa: 0.5613623018203171
Hamming Loss: 0.21686746987951808
AUC0.7795321637426901
Sensitivity0.7368421052631579
Specificity0.8222222222222222
Accuracy: 0.6746987951807228
Average Precision Score: 0.578961459874586
Kappa: 0.3539925050446815
Hamming Loss: 0.3253012048192771
AUC: 0.6795321637426901
Sensitivity0.7368421052631579
Specificity0.6222222222222222


/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)
/Users/harrisonbai/miniconda3/lib

###########################################
TRAIN_AUC ANOVA GLM: 0.795646 (0.021317)

Accuracy: 0.7439393939393939
Average Precision Score: 0.6376647547869412
Kappa: 0.48482273172353907
Hamming Loss: 0.25606060606060604
AUC: 0.7435588060588061
Sensitivity: 0.7398648648648649
Specificity: 0.7472527472527473
Accuracy: 0.7831325301204819
Average Precision Score: 0.6884908053265694
Kappa: 0.564938846825859
Hamming Loss: 0.21686746987951808
AUC0.7836257309941521
Sensitivity0.7894736842105263
Specificity0.7777777777777778
Accuracy: 0.7108433734939759
Average Precision Score: 0.6112013844853097
Kappa: 0.4292263610315187
Hamming Loss: 0.2891566265060241
AUC: 0.7190058479532164
Sensitivity0.8157894736842105
Specificity0.6222222222222222


/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defau

...

Accuracy: 0.9848484848484849
Average Precision Score: 0.9754909826338398
Kappa: 0.9693525019502953
Hamming Loss: 0.015151515151515152
AUC: 0.9843703593703595
Sensitivity: 0.9797297297297297
Specificity: 0.989010989010989
Accuracy: 0.7349397590361446
Average Precision Score: 0.6447579792855633
Kappa: 0.4571938168846611
Hamming Loss: 0.26506024096385544
AUC0.7248538011695905
Sensitivity0.6052631578947368
Specificity0.8444444444444444
Accuracy: 0.6385542168674698
Average Precision Score: 0.545582329317269
Kappa: 0.25980975029726516
Hamming Loss: 0.3614457831325301
AUC: 0.6277777777777778
Sensitivity0.5
Specificity0.7555555555555555
###########################################
TRAIN_AUC TSCR RF: 0.783975 (0.043616)

Accuracy: 0.9924242424242424
Average Precision Score: 0.9876993711739475
Kappa: 0.9846810881069539
Hamming Loss: 0.007575757575757576
AUC: 0.9921851796851797
Sensitivity: 0.9898648648648649
Specificity: 0.9945054945054945
Accuracy: 0.7108433734939759
Average Precision Score: 0.6

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)


###########################################
TRAIN_AUC ANOVA BST: 0.739116 (0.044324)



/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)


Accuracy: 0.85
Average Precision Score: 0.7644706384256224
Kappa: 0.6982097659208898
Hamming Loss: 0.15
AUC: 0.8507573507573507
Sensitivity: 0.8581081081081081
Specificity: 0.8434065934065934
Accuracy: 0.7228915662650602
Average Precision Score: 0.6231216174366991
Kappa: 0.44746743849493487
Hamming Loss: 0.27710843373493976
AUC0.7260233918128655
Sensitivity0.7631578947368421
Specificity0.6888888888888889
Accuracy: 0.6867469879518072
Average Precision Score: 0.5892970377751608
Kappa: 0.37412993039443154
Hamming Loss: 0.3132530120481928
AUC: 0.6885964912280703
Sensitivity0.7105263157894737
Specificity0.6666666666666666
###########################################
TRAIN_AUC TSCR BST: 0.748328 (0.049526)

Accuracy: 0.8636363636363636
Average Precision Score: 0.7841531370943136
Kappa: 0.7252137226601532
Hamming Loss: 0.13636363636363635
AUC: 0.8637511137511138
Sensitivity: 0.8648648648648649
Specificity: 0.8626373626373627
Accuracy: 0.7469879518072289
Average Precision Score: 0.6528489899447

In [11]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_validate = scaler.fit_transform(X_validate)
X_test = scaler.fit_transform(X_test)

# WILCOXON SCORE FUNCTION
def takeSecond(elem):
    return elem[1]
def WLCX(data, target, n_selected_features):
    pval = []
    for num in range(len(data[1])):
        x = data[:,num]
        pval.append([num, wilcoxon(x,target)[1]])
    pval.sort(key=takeSecond)
    idx = []
    for i in range(n_selected_features):
        idx.append(pval[i][0])
    return idx

# MULTIVARIATE FEATURE SELECTION X CLASSIFICATION (10 fold CV)

# print('BEFORE')
MV_sel = []
MV_sel.append(('WLCX', WLCX(X_train, Y_train, n_selected_features=num_fea)))
print('WLCX')
for name, model in models:
    for kind, idx in MV_sel:
        # X_sel = X[:, idx[0:num_fea]]
        X_test_ = X_test[:,idx[0:num_fea]]
        X_validate_ = X_validate[:,idx[0:num_fea]]
        X_train_ = X_train[:, idx[0:num_fea]]
        # X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X_sel, Y, test_size=validation_size, random_state=seed)
        #kfold = model_selection.KFold(n_splits=10, random_state=seed)
        cv_results = model_selection.cross_val_score(model, X_train_, Y_train, cv=kfold, scoring='roc_auc')
        
        sheet.write(r,c,cv_results.mean())
        
        msg = "%s %s: %f (%f)\n" % (kind, name, cv_results.mean(), cv_results.std())
        print(msg)
        output.write(msg)
        model.fit(X_train_, Y_train)
        joblib.dump(model,'./handpkl/Endo'+name+kind+'100.pkl')
        
        Y_pred = model.predict(X_train_)
        print("Accuracy: " + repr(accuracy_score(Y_train, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_train, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_train, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_train, Y_pred)))
        print("AUC: " + repr(roc_auc_score(Y_train, Y_pred)))
        print("Sensitivity: " + repr(recall_score(Y_train, Y_pred)))
        tn, fp, fn, tp = confusion_matrix(Y_train, Y_pred).ravel()
        print("Specificity: " + repr(tn / (tn + fp)))
        
        sheet_train.write(r,c,roc_auc_score(Y_train, Y_pred))
        
        Y_pred = model.predict(X_validate_)    
        print("Accuracy: " + repr(accuracy_score(Y_validate, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_validate, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_validate, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_validate, Y_pred)))
        print("AUC"+repr(roc_auc_score(Y_validate,Y_pred)))
        print("Sensitivity" + repr(recall_score(Y_validate,Y_pred)))
        tn,fp,fn,tp = confusion_matrix(Y_validate,Y_pred).ravel()
        print("Specificity" + repr(tn/(tn+fp)))
        
        sheet_validate.write(r,c,roc_auc_score(Y_validate,Y_pred))
        
        Y_pred = model.predict(X_test_)
        print("Accuracy: " + repr(accuracy_score(Y_test, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_test, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_test, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_test, Y_pred)))
        print("AUC"+repr(roc_auc_score(Y_test,Y_pred)))
        print("Sensitivity" + repr(recall_score(Y_test,Y_pred)))
        tn,fp,fn,tp = confusion_matrix(Y_test,Y_pred).ravel()
        print("Specificity" + repr(tn/(tn+fp)))
        
        sheet_test.write(r,c,roc_auc_score(Y_test,Y_pred))
        
        r = r + 1
    c = c + 1
    r = 0

WLCX
WLCX GLM: 0.782884 (0.047408)

Accuracy: 0.7272727272727273
Average Precision Score: 0.6237661752661753
Kappa: 0.44057261254473534
Hamming Loss: 0.2727272727272727
AUC: 0.7170886545886546
Sensitivity: 0.6182432432432432
Specificity: 0.8159340659340659
Accuracy: 0.7349397590361446
Average Precision Score: 0.6476583023824621
Kappa: 0.45492537313432835
Hamming Loss: 0.26506024096385544
AUC0.7228070175438597
Sensitivity0.5789473684210527
Specificity0.8666666666666667
Accuracy: 0.7469879518072289
Average Precision Score: 0.6528489899447414
Kappa: 0.4872021182700794
Hamming Loss: 0.25301204819277107
AUC0.7421052631578947
Sensitivity0.6842105263157895
Specificity0.8


/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defau

WLCX LDA: 0.737468 (0.036810)

Accuracy: 0.7893939393939394
Average Precision Score: 0.6963681949531006
Kappa: 0.5700627987627707
Hamming Loss: 0.2106060606060606
AUC: 0.7822430947430947
Sensitivity: 0.7128378378378378
Specificity: 0.8516483516483516
Accuracy: 0.7108433734939759
Average Precision Score: 0.6157557863031071
Kappa: 0.4103019538188277
Hamming Loss: 0.2891566265060241
AUC0.7026315789473684
Sensitivity0.6052631578947368
Specificity0.8
Accuracy: 0.6024096385542169
Average Precision Score: 0.5176193495787662
Kappa: 0.19417475728155342
Hamming Loss: 0.39759036144578314
AUC0.5964912280701755
Sensitivity0.5263157894736842
Specificity0.6666666666666666
WLCX KNN: 0.710515 (0.062731)



/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


Accuracy: 0.7666666666666667
Average Precision Score: 0.663647624897625
Kappa: 0.5289210233592881
Hamming Loss: 0.23333333333333334
AUC: 0.7647943272943272
Sensitivity: 0.7466216216216216
Specificity: 0.782967032967033
Accuracy: 0.7469879518072289
Average Precision Score: 0.655031609692358
Kappa: 0.48508124076809456
Hamming Loss: 0.25301204819277107
AUC0.7400584795321637
Sensitivity0.6578947368421053
Specificity0.8222222222222222
Accuracy: 0.7349397590361446
Average Precision Score: 0.6373777659112906
Kappa: 0.4660818713450292
Hamming Loss: 0.26506024096385544
AUC0.7330409356725145
Sensitivity0.7105263157894737
Specificity0.7555555555555555
WLCX DT: 0.613344 (0.016899)

Accuracy: 0.9924242424242424
Average Precision Score: 0.9906838656838657
Kappa: 0.9846617212337554
Hamming Loss: 0.007575757575757576
AUC: 0.9915540540540541
Sensitivity: 0.9831081081081081
Specificity: 1.0
Accuracy: 0.6987951807228916
Average Precision Score: 0.6011499768633567
Kappa: 0.39203047172575445
Hamming Loss: 

...

WLCX SVM: 0.776773 (0.041707)

Accuracy: 0.7045454545454546
Average Precision Score: 0.6037296037296037
Kappa: 0.3830888697152718
Hamming Loss: 0.29545454545454547
AUC: 0.6854395604395604
Sensitivity: 0.5
Specificity: 0.8708791208791209
Accuracy: 0.6746987951807228
Average Precision Score: 0.5807675145024542
Kappa: 0.3296440323063117
Hamming Loss: 0.3253012048192771
AUC0.6611111111111111
Sensitivity0.5
Specificity0.8222222222222222
Accuracy: 0.7228915662650602
Average Precision Score: 0.6346438385119425
Kappa: 0.42895602752019146
Hamming Loss: 0.27710843373493976
AUC0.7096491228070176
Sensitivity0.5526315789473685
Specificity0.8666666666666667


/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


WLCX BAG: 0.720545 (0.038628)

Accuracy: 0.9696969696969697
Average Precision Score: 0.9597534222534223
Kappa: 0.9384328358208955
Hamming Loss: 0.030303030303030304
AUC: 0.9668473418473418
Sensitivity: 0.9391891891891891
Specificity: 0.9945054945054945
Accuracy: 0.7349397590361446
Average Precision Score: 0.6511755524120775
Kappa: 0.4526378896882495
Hamming Loss: 0.26506024096385544
AUC0.7207602339181287
Sensitivity0.5526315789473685
Specificity0.8888888888888888
Accuracy: 0.6746987951807228
Average Precision Score: 0.5783475295206431
Kappa: 0.34339290946381484
Hamming Loss: 0.3253012048192771
AUC0.671345029239766
Sensitivity0.631578947368421
Specificity0.7111111111111111


/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the op

WLCX NNet: 0.782835 (0.053985)

Accuracy: 0.7136363636363636
Average Precision Score: 0.612100448720167
Kappa: 0.4055584148224395
Hamming Loss: 0.2863636363636364
AUC: 0.6974680724680724
Sensitivity: 0.5405405405405406
Specificity: 0.8543956043956044
Accuracy: 0.7469879518072289
Average Precision Score: 0.6645059771248737
Kappa: 0.4786120251271313
Hamming Loss: 0.25301204819277107
AUC0.7339181286549707
Sensitivity0.5789473684210527
Specificity0.8888888888888888
Accuracy: 0.7710843373493976
Average Precision Score: 0.6871867776709554
Kappa: 0.5321862948679917
Hamming Loss: 0.2289156626506024
AUC0.762280701754386
Sensitivity0.6578947368421053
Specificity0.8666666666666667
WLCX RF: 0.743516 (0.061208)

Accuracy: 0.9651515151515152
Average Precision Score: 0.945772723021858
Kappa: 0.9293992893419902
Hamming Loss: 0.03484848484848485
AUC: 0.9636731511731511
Sensitivity: 0.9493243243243243
Specificity: 0.978021978021978
Accuracy: 0.7108433734939759
Average Precision Score: 0.6144951322317133

In [12]:
MV_sel = []
MV_sel.append(('MIM', MIM.mim(X_train, Y_train, n_selected_features=num_fea)))
print('MIM')
MV_sel.append(('MIFS', MIFS.mifs(X_train, Y_train, n_selected_features=num_fea)))
print('MIFS')
MV_sel.append(('MRMR', MRMR.mrmr(X_train, Y_train, n_selected_features=num_fea)))
print('MRMR')
MV_sel.append(('CIFE', CIFE.cife(X_train, Y_train, n_selected_features=num_fea)))
print('CIFE')
MV_sel.append(('JMI', JMI.jmi(X_train, Y_train, n_selected_features=num_fea)))
print('JMI')
MV_sel.append(('CMIM', CMIM.cmim(X_train, Y_train, n_selected_features=num_fea)))
print('CMIM')
MV_sel.append(('ICAP', ICAP.icap(X_train, Y_train, n_selected_features=num_fea)))
print('ICAP')
MV_sel.append(('DISR', DISR.disr(X_train, Y_train, n_selected_features=num_fea)))

MIM
MIFS
MRMR
CIFE
JMI
CMIM
ICAP


In [13]:
for name, model in models:
    for kind, idx in MV_sel:
        #print(idx[0:num_fea][0])
        # X_sel = X[:, idx[0:num_fea]]
        X_test_ = X_test[:,idx[0:num_fea][0]]
        X_validate_ = X_validate[:,idx[0:num_fea][0]]
        X_train_ = X_train[:, idx[0:num_fea][0]]
        # X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X_sel, Y, test_size=validation_size, random_state=seed)
        #kfold = model_selection.KFold(n_splits=10, random_state=seed)
        cv_results = model_selection.cross_val_score(model, X_train_, Y_train, cv=kfold, scoring='roc_auc')
        
        sheet.write(r,c,cv_results.mean())
        
        msg = "%s %s: %f (%f)\n" % (kind, name, cv_results.mean(), cv_results.std())
        print(msg)
        output.write(msg)
        model.fit(X_train_, Y_train)
        joblib.dump(model,'./handpkl/Endo'+name+kind+'100.pkl')
        
        Y_pred = model.predict(X_train_)
        print("Accuracy: " + repr(accuracy_score(Y_train, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_train, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_train, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_train, Y_pred)))
        print("AUC: " + repr(roc_auc_score(Y_train, Y_pred)))
        print("Sensitivity: " + repr(recall_score(Y_train, Y_pred)))
        tn, fp, fn, tp = confusion_matrix(Y_train, Y_pred).ravel()
        print("Specificity: " + repr(tn / (tn + fp)))
        
        sheet_train.write(r,c,roc_auc_score(Y_train, Y_pred))
        
        Y_pred = model.predict(X_validate_)    
        print("Accuracy: " + repr(accuracy_score(Y_validate, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_validate, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_validate, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_validate, Y_pred)))
        print("AUC"+repr(roc_auc_score(Y_validate,Y_pred)))
        print("Sensitivity" + repr(recall_score(Y_validate,Y_pred)))
        tn,fp,fn,tp = confusion_matrix(Y_validate,Y_pred).ravel()
        print("Specificity" + repr(tn/(tn+fp)))
        
        sheet_validate.write(r,c,roc_auc_score(Y_validate,Y_pred))
        
        Y_pred = model.predict(X_test_)
        print("Accuracy: " + repr(accuracy_score(Y_test, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_test, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_test, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_test, Y_pred)))
        print("AUC"+repr(roc_auc_score(Y_test,Y_pred)))
        print("Sensitivity" + repr(recall_score(Y_test,Y_pred)))
        tn,fp,fn,tp = confusion_matrix(Y_test,Y_pred).ravel()
        print("Specificity" + repr(tn/(tn+fp)))
        
        sheet_test.write(r,c,roc_auc_score(Y_test,Y_pred))
        
        r = r + 1
    c = c + 1
    r = 0

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defau

MIM GLM: 0.802764 (0.025179)

Accuracy: 0.75
Average Precision Score: 0.6456565515843866
Kappa: 0.4938368007139272
Hamming Loss: 0.25
AUC: 0.7465288090288089
Sensitivity: 0.7128378378378378
Specificity: 0.7802197802197802
Accuracy: 0.7590361445783133
Average Precision Score: 0.6883719086873811
Kappa: 0.5003010234798314
Hamming Loss: 0.24096385542168675
AUC0.7429824561403511
Sensitivity0.5526315789473685
Specificity0.9333333333333333
Accuracy: 0.7710843373493976
Average Precision Score: 0.6780921695316116
Kappa: 0.5379431585115734
Hamming Loss: 0.2289156626506024
AUC0.768421052631579
Sensitivity0.7368421052631579
Specificity0.8
MIFS GLM: 0.833839 (0.040891)

Accuracy: 0.7878787878787878
Average Precision Score: 0.6909033232976896
Kappa: 0.5695759111574867
Hamming Loss: 0.21212121212121213
AUC: 0.7837095337095337
Sensitivity: 0.7432432432432432
Specificity: 0.8241758241758241
Accuracy: 0.7831325301204819
Average Precision Score: 0.6832386755259022
Kappa: 0.5701956271576525
Hamming Loss: 

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defau

Accuracy: 0.7848484848484848
Average Precision Score: 0.6859600961484523
Kappa: 0.5645305181770872
Hamming Loss: 0.21515151515151515
AUC: 0.7819089694089695
Sensitivity: 0.7533783783783784
Specificity: 0.8104395604395604
Accuracy: 0.8192771084337349
Average Precision Score: 0.7295420449448629
Kappa: 0.6381865736704446
Hamming Loss: 0.18072289156626506
AUC0.8210526315789475
Sensitivity0.8421052631578947
Specificity0.8
Accuracy: 0.6265060240963856
Average Precision Score: 0.5487927418174723
Kappa: 0.28706012745913
Hamming Loss: 0.37349397590361444
AUC0.6514619883040935
Sensitivity0.9473684210526315
Specificity0.35555555555555557
JMI GLM: 0.814337 (0.028632)

Accuracy: 0.7560606060606061
Average Precision Score: 0.6519246519246519
Kappa: 0.5070422535211268
Hamming Loss: 0.24393939393939393
AUC: 0.7536011286011286
Sensitivity: 0.7297297297297297
Specificity: 0.7774725274725275
Accuracy: 0.6626506024096386
Average Precision Score: 0.5839297037775161
Kappa: 0.2853628536285362
Hamming Loss: 0

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defau

...

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the op

ICAP NNet: 0.802430 (0.029512)

Accuracy: 0.7560606060606061
Average Precision Score: 0.6502647631957976
Kappa: 0.5104398945874722
Hamming Loss: 0.24393939393939393
AUC: 0.7570723195723196
Sensitivity: 0.7668918918918919
Specificity: 0.7472527472527473
Accuracy: 0.7349397590361446
Average Precision Score: 0.6373777659112906
Kappa: 0.4660818713450292
Hamming Loss: 0.26506024096385544
AUC0.7330409356725145
Sensitivity0.7105263157894737
Specificity0.7555555555555555
Accuracy: 0.6987951807228916
Average Precision Score: 0.6006689295883223
Kappa: 0.40900028481913986
Hamming Loss: 0.30120481927710846
AUC0.7099415204678361
Sensitivity0.8421052631578947
Specificity0.5777777777777777


/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the op

DISR NNet: 0.794927 (0.042654)

Accuracy: 0.7515151515151515
Average Precision Score: 0.6457750582750583
Kappa: 0.5002216312056738
Hamming Loss: 0.24848484848484848
AUC: 0.7513736263736264
Sensitivity: 0.75
Specificity: 0.7527472527472527
Accuracy: 0.7831325301204819
Average Precision Score: 0.6908437072389281
Kappa: 0.5631578947368421
Hamming Loss: 0.21686746987951808
AUC0.781578947368421
Sensitivity0.7631578947368421
Specificity0.8
Accuracy: 0.6626506024096386
Average Precision Score: 0.5689945135232004
Kappa: 0.33141542002301494
Hamming Loss: 0.3373493975903614
AUC0.6684210526315789
Sensitivity0.7368421052631579
Specificity0.6
MIM RF: 0.757842 (0.043285)

Accuracy: 0.9833333333333333
Average Precision Score: 0.9795045045045045
Kappa: 0.9661916736518581
Hamming Loss: 0.016666666666666666
AUC: 0.9814189189189189
Sensitivity: 0.9628378378378378
Specificity: 1.0
Accuracy: 0.5301204819277109
Average Precision Score: 0.46374514074605055
Kappa: 0.02352941176470591
Hamming Loss: 0.469879518

JMI BST: 0.766457 (0.031003)

Accuracy: 0.8727272727272727
Average Precision Score: 0.7979601591356558
Kappa: 0.7432096935561567
Hamming Loss: 0.12727272727272726
AUC: 0.8723084348084349
Sensitivity: 0.8682432432432432
Specificity: 0.8763736263736264
Accuracy: 0.5662650602409639
Average Precision Score: 0.5080205691185796
Kappa: 0.1704608550805108
Hamming Loss: 0.43373493975903615
AUC0.589766081871345
Sensitivity0.868421052631579
Specificity0.3111111111111111
Accuracy: 0.6385542168674698
Average Precision Score: 0.5569911223842738
Kappa: 0.30871737923375897
Hamming Loss: 0.3614457831325301
AUC0.6625730994152046
Sensitivity0.9473684210526315
Specificity0.37777777777777777
CMIM BST: 0.755198 (0.046504)

Accuracy: 0.8590909090909091
Average Precision Score: 0.7845251950515109
Kappa: 0.7141659681475272
Hamming Loss: 0.1409090909090909
AUC: 0.8558434808434808
Sensitivity: 0.8243243243243243
Specificity: 0.8873626373626373
Accuracy: 0.6626506024096386
Average Precision Score: 0.5695082963432

In [14]:
book.save(file)
output.close()